### Lab 7: Cifrado de extremo a extremo (E2EE)

#### TEL252 - Criptografía y Seguridad en la Información
##### 2do Semestre, 2025
##### Docente: Daniel Espinoza
##### Fecha de entrega: 27 de Noviembre, 08:00 horas

_Nota_: El uso de herramientas de Inteligencia Artificial (IA) está permitido únicamente si se justifica adecuadamente y se evidencia comprensión del contenido. En caso de detectarse el uso de IA sin justificación, o la inclusión de conceptos no abordados en la asignatura sin una explicación clara, el laboratorio será evaluado con nota mínima.

##### **Actividad 1** (100%) 

En este laboratorio usted y su grupo tendrá que desarrollar una API básica con Flask y Python. Para ello, puede apoyarse en las APIs presentadas en los laboratorios 2 y 5, donde se tiene un script `server.py` que define las rutas y corre el servidor, además de un archivo `crypto.py` que describe las funcionalidades criptográficas. No es necesario crear un docker, y puede modificar la estructura a voluntad.

Lo que se busca en el último laboratorio de la asignatura es que ustedes puedan **integrar** primitivas criptográficas para crear una API básica funcional que implemente cifrado de extremo a extremo (E2EE), es decir, que el cliente y el servidor puedan comunicarse de forma segura. Para distinguir entre grupos, cada uno tiene que elegir un contexto entre los siguientes:

1. Redes de Sensores (IoT)
2. Chat (_WhatsApp_ por ejemplo)
3. Gestor de contraseñas
4. Pasarela de pago
5. Voto electrónico

Puede proponer otros temas si prefiere. El **registro de los grupos** debe realizarse a más tardar el **Jueves 23 de Octubre, a las 23:59 horas**.




En este laboratorio no se les va a indicar cómo realizar la API. Cada grupo debe tomar sus propias **decisiones de diseño que deben tener sentido con el contexto elegido**. Se vuelve a recalcar que la API puede ser simple, por lo que aspectos como bases de datos, configuraciones, etc, se pueden simplificar a conveniencia, lo importante es que en este Lab aprendan a **integrar**. Para esta evaluación, el rol del profesor será el de una guía.

La entrega del Lab 7 se divide en dos partes:

1. API (50%): Corresponde a los archivos necesarios para que la API sea funcional. Recordar que **debe** ser una API en Python con Flask.
2. Diagrama (50%): Cada grupo debe realizar un diagrama en formato libre (puede ser UML, _Flowchart_, _Timing_, etc) que debe describir cómo funciona la comunicación con la API. En el diagrama **se deben especificar los algoritmos utilizados para cada proceso, sus parámetros y la matemática involucrada en ellos**. Dicho de otra manera, el diagrama debe ser autocontenido, es decir, mediante el diagrama debe ser suficiente para entender la API al completo.

Durante el Jueves 27 de Noviembre, todos los grupos deberán llevar impreso su diagrama para pegarlo en la pizarra de la sala. Los dos bloques serán utilizados para que los grupos puedan analizar los diagramas de sus otros compañeros y pegar críticas constructivas y sugerencias a través de adhesivos _Post-it_. **La exposición es el requisito para que su diagrama sea corregido por el profesor**.

In [3]:
# ==============================================================================
#                 CÓDIGO COMPLETO PARA CELDA DEL NOTEBOOK
# ==============================================================================

import requests
import json
import base64
import os
import random

# Importar funciones de PyCryptodome para criptografía
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
# Para asegurar la compatibilidad con las funciones del servidor
from Crypto.Util.number import bytes_to_long, long_to_bytes 

# Parámetros Globales (Deben coincidir con crypto.py)
RSA_KEY_SIZE = 2048 # 2048 bits
AES_KEY_SIZE = 16   # 128 bits

# Configuración del Servidor
HOST = "http://127.0.0.1:5000"

# --- Funciones de Utilidad (Asegura que estén definidas en tu notebook) ---

def base64_encode(data):
    """Codifica bytes a string Base64."""
    return base64.b64encode(data).decode('utf-8')

def base64_decode(data):
    """Decodifica string Base64 a bytes."""
    return base64.b64decode(data)

def bytes_to_int(data):
    """Convierte bytes a entero (Big Endian)."""
    return bytes_to_long(data)

def int_to_bytes(data):
    """Convierte entero a bytes (Big Endian) con el tamaño correcto para RSA."""
    # Asegura que el resultado sea siempre de 2048 bits / 8 = 256 bytes
    return long_to_bytes(data, RSA_KEY_SIZE // 8)

# --- Función Principal de Votación (Cifrado Híbrido Seguro) ---

def cast_secure_vote(candidate_name):
    print(f"--- Iniciando Votación para el Candidato: {candidate_name} ---")
    
    # 1. Obtener la llave pública del servidor
    print("1. Obteniendo llave pública del servidor...")
    response = requests.get(f"{HOST}/public_key").json()
    
    # Parámetros RSA del Servidor
    N_bytes = base64_decode(response["modulus"])
    N = bytes_to_int(N_bytes) # Módulo
    E = response["exponent"]   # Exponente Público
    
    # 2. GENERACIÓN Y CIFRADO DE LA LLAVE DE SESIÓN (E2EE Real)
    print("2. Generando K_sess AES y cifrándola con RSA-OAEP (Llave Pública del Servidor)...")
    
    # a) Generar la llave de sesión AES (K_sess) aleatoria y efímera
    session_key = get_random_bytes(AES_KEY_SIZE) 
    
    # b) Cifrar K_sess usando la llave pública RSA del servidor (N, E)
    # Creamos el objeto llave pública para usar OAEP
    recipient_key = RSA.construct((N, E))
    cipher_rsa = PKCS1_OAEP.new(recipient_key)
    encrypted_aes_key = cipher_rsa.encrypt(session_key) # K_sess cifrada con RSA-OAEP
    
    # 3. CIFRADO DEL VOTO CON LA LLAVE DE SESIÓN (AES-EAX)
    print("3. Cifrando el voto con K_sess (AES-EAX)...")
    vote_data = {"candidate": candidate_name, "timestamp": "2025-11-09T23:59:00Z"}
    data_bytes = json.dumps(vote_data).encode('utf-8')
    
    # Cifrado del mensaje con la K_sess generada
    cipher_aes = AES.new(session_key, AES.MODE_EAX)
    nonce = cipher_aes.nonce
    ciphertext, tag = cipher_aes.encrypt_and_digest(data_bytes)

    # El mensaje a cegar y firmar es la concatenación de los componentes cifrados (nonce+ciphertext+tag)
    message_to_sign = nonce + ciphertext + tag
    
    # 4. Generar Factor de Cegado (Blind Factor)
    print("4. Generando factor de cegado aleatorio (blind factor)...")
    r_int = bytes_to_int(get_random_bytes(RSA_KEY_SIZE // 8))
    # Aseguramos r < N
    r_int = r_int % N
    
    # 5. Aplicar Cegado (Blinding)
    print("5. Aplicando cegado al mensaje (Blind Signature)...")
    
    m_int = bytes_to_int(message_to_sign)
    
    # blinded_msg = m * r^E mod N
    blind_factor_pow_E = pow(r_int, E, N)
    blinded_msg_int = (m_int * blind_factor_pow_E) % N
    blinded_msg_bytes = int_to_bytes(blinded_msg_int)
    
    # 6. Solicitud de Firma Ciega al Servidor
    print("6. Solicitando firma ciega al servidor...")
    blind_sig_request = {
        "blinded_hash": base64_encode(blinded_msg_bytes)
    }
    response = requests.post(f"{HOST}/request_signature", json=blind_sig_request).json()
    
    if "error" in response:
        print(f"ERROR en firma: {response}")
        return

    blind_signature_bytes = base64_decode(response["blind_signature"])
    blind_signature_int = bytes_to_int(blind_signature_bytes)
    
    # 7. Descegado de la Firma
    print("7. Descegando la firma...")
    # Calculamos el inverso multiplicativo modular de r
    r_inv = pow(r_int, -1, N) 
    # signature = blind_signature * r^-1 mod N
    unblinded_signature_int = (blind_signature_int * r_inv) % N
    unblinded_signature_bytes = int_to_bytes(unblinded_signature_int)
    
    # 8. Emisión del Voto
    print("8. Emitiendo el voto cifrado, firma descegada y llave de sesión encapsulada...")
    
    # El blind_factor es la llave única que verifica la unicidad del voto en el servidor
    blind_factor_hex = r_int.to_bytes(RSA_KEY_SIZE // 8, 'big').hex()
    
    vote_payload = {
        # Componentes AES para descifrado y verificación
        "nonce": base64_encode(nonce),
        "ciphertext": base64_encode(ciphertext),
        "tag": base64_encode(tag),
        
        # Llave de sesión cifrada con RSA-OAEP (Permite al servidor descifrar la K_sess)
        "encrypted_aes_key": base64_encode(encrypted_aes_key), 
        
        # Componentes de firma ciega para verificación de unicidad y validez
        "signature": base64_encode(unblinded_signature_bytes),
        "blind_factor": blind_factor_hex
    }
    
    response = requests.post(f"{HOST}/vote", json=vote_payload).json()
    
    print("\n--- Resultado de la Votación ---")
    print(json.dumps(response, indent=2))
    print("-------------------------------")


# ---------------------------------------------------------------------------
# --- Ejecución del Programa ---
# Para ejecutar, primero asegúrate de correr 'python server.py' en un terminal
# antes de ejecutar esta celda y haber realizado las correcciones en crypto.py y server.py.

# Voto 1
cast_secure_vote("Candidate X")
# Voto 2 (Diferente)
cast_secure_vote("Candidate Y")
# Intento de Voto 3 (Mismo candidato/firma, pero con un nuevo blind factor)
cast_secure_vote("Candidate X")

--- Iniciando Votación para el Candidato: Candidate X ---
1. Obteniendo llave pública del servidor...
2. Generando K_sess AES y cifrándola con RSA-OAEP (Llave Pública del Servidor)...
3. Cifrando el voto con K_sess (AES-EAX)...
4. Generando factor de cegado aleatorio (blind factor)...
5. Aplicando cegado al mensaje (Blind Signature)...
6. Solicitando firma ciega al servidor...
7. Descegando la firma...
8. Emitiendo el voto cifrado, firma descegada y llave de sesión encapsulada...

--- Resultado de la Votación ---
{
  "message": "Vote successfully cast and recorded.",
  "vote_id": "9b297247dd98980ac471b4ae6e1dc401569341ce2b1a60be856302c38af2786432004c702db5d24ddff2787cbec3e9f05b9c93fd6b778c704afb110959b519ddbd419cc90d033fcc27ac9ec878a2c91e66e105d3f609d3ed4462974668ac1a455e0c054c60989defbe3f3ba7630d687611d3753b0ecfc7c60ebee6a4fbd0ddf78d8a44de7659398ef4f1189d87d340146dbb4c0ebc9381cd1b22b058f2dc61ad9ed450319e24252fff2b337823cc95b81d7845d51860190ecd4011523a4f05cd3963c76dd3a50ce1ec2d25f92a92